#### RandonForest with LogisticRegression coefficient

In [1]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# NLP
import nltk
from soynlp.normalizer import *
from hanspell import spell_checker
from konlpy.tag import Okt

# ML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score , recall_score, f1_score, roc_auc_score, roc_curve

#random
import random

#wordcloud
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image

# 한글화
import matplotlib.pyplot as plt 
from matplotlib import rc
plt.rcParams['axes.unicode_minus'] = False #마이너스 부호 때문에 한글이 깨질 수가 있어 주는 설정
rc('font', family='Arial Unicode MS')
%matplotlib inline 

#time
import time

In [2]:
df_review = pd.read_csv('review_all_morph_tknized_rmved_v2.csv')
df_review = df_review[['date', 
          'tokenized_review', 
          'tokenized_removed_review', 
          'reviews_for_vectorize', 
          'noun_tokenized_review', 
          'noun_tokenized_removed_review', 
          'noun_reviews_for_vectorize', 
          'rating', 'label']]

df_review

,date,tokenized_review,tokenized_removed_review,reviews_for_vectorize,noun_tokenized_review,noun_tokenized_removed_review,noun_reviews_for_vectorize,rating,label
0,2023-09-10,"['상세', '설명', '에', '들어가지', '않아도', '검색', '결과', '...","['상세', '설명', '들어가지', '않아도', '검색', '결과', '중량', ...",상세 설명 들어가지 않아도 검색 결과 중량 가격 확인 되면 더 편리하겠습니다,"['상세', '설명', '검색', '결과', '중량', '가격', '바로', '확인...","['상세', '설명', '검색', '결과', '중량', '가격', '확인', '더']",상세 설명 검색 결과 중량 가격 확인 더,5,1.0
1,2023-09-09,"['필터', '기능', '잘', '되어있어', '편하게', '쇼핑', '함']","['필터', '기능', '잘', '되어있어', '편하게', '쇼핑', '함']",필터 기능 잘 되어있어 편하게 쇼핑 함,"['필터', '기능', '쇼핑', '함']","['필터', '기능', '쇼핑', '함']",필터 기능 쇼핑 함,5,1.0
2,2023-09-09,"['배송', '이', '빨라요', '그리고', '할인', '이', '많아요']","['배송', '빨라요', '할인', '많아요']",배송 빨라요 할인 많아요,"['배송', '할인']","['배송', '할인']",배송 할인,5,1.0
3,2023-09-09,"['50', '퍼', '쿠폰', '만원', '빵빵', '하게', '주는', '곳',...","['50', '퍼', '쿠폰', '만원', '빵빵', '하게', '주는', '곳',...",50 퍼 쿠폰 만원 빵빵 하게 주는 곳 처음 봄 주문 하면 6시간 만에 옴 앱 뭐,"['쿠폰', '만원', '빵빵', '곳', '처음', '봄', '주문', '옴', ...","['쿠폰', '만원', '빵빵', '곳', '처음', '봄', '주문', '옴', ...",쿠폰 만원 빵빵 곳 처음 봄 주문 옴 앱 뭐,5,1.0
4,2023-09-09,"['빠른', '배송']","['빠른', '배송']",빠른 배송,['배송'],['배송'],배송,5,1.0
...,...,...,...,...,...,...,...,...,...
11224,2016-03-02,"['모바일', '로', '쉽게', '살', '수', '있어서']","['모바일', '쉽게', '살', '있어서']",모바일 쉽게 살 있어서,"['모바일', '살', '수']","['모바일', '살']",모바일 살,5,1.0
11225,2016-03-02,"['아이', '를', '키우며', '좋은', '식', '자재', '에', '대한',...","['키우며', '좋은', '식', '자재', '대한', '갈증', '있는데', '이...",키우며 좋은 식 자재 대한 갈증 있는데 이렇게 건강하고 신선한 먹거리 쉽게 접 있...,"['아이', '식', '자재', '대한', '갈증', '먹거리', '접', '수',...","['식', '자재', '대한', '갈증', '먹거리', '접', '컴퓨터', '엄마...",식 자재 대한 갈증 먹거리 접 컴퓨터 엄마 앱 지금 처럼 식탁,5,1.0
11226,2016-03-02,"['아는', '동생', '소개', '로', '알', '게', '된', '마켓', '...","['아는', '동생', '소개', '알', '게', '된', '마켓', '컬리', ...",아는 동생 소개 알 게 된 마켓 컬리 늦게 게 아쉬울 정도 제품 퀄리티 완젼 좋고...,"['동생', '소개', '알', '마켓', '컬리', '안', '게', '정도', ...","['동생', '소개', '알', '마켓', '컬리', '게', '정도', '제품',...",동생 소개 알 마켓 컬리 게 정도 제품 퀄리티 완젼 포장 배송 눈 안보 사도 정말...,5,1.0
11227,2016-03-02,"['샛별', '배송', '정말', '훌륭합니다', '컬리', '라면', '무조건',...","['샛별', '배송', '정말', '훌륭합니다', '컬리', '라면', '무조건',...",샛별 배송 정말 훌륭합니다 컬리 라면 무조건 믿고 주문,"['샛별', '배송', '정말', '컬리', '라면', '무조건', '주문']","['샛별', '배송', '정말', '컬리', '라면', '무조건', '주문']",샛별 배송 정말 컬리 라면 무조건 주문,5,1.0


---

#### morph

In [3]:
#vectorize reviews
contents_for_vectorize_morph = df_review['reviews_for_vectorize']

#TfidfVectorizer
vectorizer_tfidf_morph = TfidfVectorizer(min_df=7, decode_error='ignore')

vectorizer_tfidf_morph.fit_transform(contents_for_vectorize_morph)

#말뭉치
corpus_morph = []

for word in vectorizer_tfidf_morph.get_feature_names_out():
    corpus_morph.append(word)

In [4]:
#RandomForest

start = time.time()

#feature, label 분리
X_morph = vectorizer_tfidf_morph.fit_transform(contents_for_vectorize_morph)
y_morph = df_review['label']

#train, test data 분리
X_morph_train, X_morph_test, y_morph_train, y_morph_test = train_test_split(X_morph, 
                                                                            y_morph, 
                                                                            test_size=0.3, 
                                                                            random_state=13, 
                                                                            stratify=y_morph)



#RandomForestClassifier
rdmforest_clf_morph = RandomForestClassifier(random_state=13, n_jobs=-1)

#GridSearch
params = {
    'max_depth' : [35],
    'n_estimators' : [400, 500],
    'min_samples_leaf' : [1], 
    'min_samples_split' : [6],
    'max_features' : [0.1]
}

grid_cv = GridSearchCV(estimator=rdmforest_clf_morph, 
                       param_grid=params, 
                       cv=5, 
                       n_jobs=-1)

grid_cv.fit(X_morph_train, y_morph_train)

print(start - time.time())

-10.975695848464966


In [5]:
grid_cv.best_estimator_

RandomForestClassifier(max_depth=35, max_features=0.1, min_samples_split=6,
                       n_estimators=500, n_jobs=-1, random_state=13)

In [6]:
#고려한 parameter
#'max_depth' : [20, 25, 30, 35]
#'n_estimators' : [100, 200, 300, 400, 450, 500]
#'min_samples_leaf' : [6, 8, 10, 12, 14]
#'min_samples_split' : [6,8]
#'max_features' : [0.1, 0.2,0.3, 0.5]

grid_cv.best_params_

{'max_depth': 35,
 'max_features': 0.1,
 'min_samples_leaf': 1,
 'min_samples_split': 6,
 'n_estimators': 500}

In [7]:
grid_cv.best_score_

0.8931297709923663

In [8]:
#X_test predict

#skfold
skfold = StratifiedKFold(n_splits=5)

#best randomforest
rdmforest_clf_morph = RandomForestClassifier(max_depth=35, 
                                             max_features=0.1, 
                                             min_samples_leaf=1, 
                                             min_samples_split=6, 
                                             n_estimators=500)
                                         

#평가지표
accuracy = []
f1 = []
auc = []


#test_data 평가지표
accuracy_test = []
f1_test = []
auc_test = []


for train_idx, validation_idx in skfold.split(X_morph_train, y=y_morph_train):
        
    X_train = X_morph_train[train_idx]
    X_validation = X_morph_train[validation_idx]
        
    y_train = y_morph_train.iloc[train_idx]
    y_validation = y_morph_train.iloc[validation_idx]
        
    #fit_predict
    rdmforest_clf_morph.fit(X_train, y_train)
    pred_validation = rdmforest_clf_morph.predict(X_validation)
        
        
    #accuracy, f1, auc
    accuracy.append(accuracy_score(pred_validation, y_validation))
    f1.append(f1_score(pred_validation, y_validation))
    auc.append(roc_auc_score(pred_validation, y_validation))

    
pred_test = rdmforest_clf_morph.predict(X_morph_test) 
        
#test_data accuracy, f1, auc
accuracy_test.append(accuracy_score(pred_test, y_morph_test))
f1_test.append(f1_score(pred_test, y_morph_test))
auc_test.append(roc_auc_score(pred_test, y_morph_test))
    
#평가지표

print('accuracy of sag : {}'.format(sum(accuracy) / len(accuracy)))
print('-'*100)
print('f1 of sag : {}'.format(sum(f1) / len(f1)))
print('-'*100)
print('auc of sag : {}'.format(sum(auc) / len(auc)))

print('='*100)


print('accuracy_test of sag : {}'.format(accuracy_test))
print('-'*100)
print('f1_test of sag : {}'.format(f1_test))
print('-'*100)
print('auc_test of sag : {}'.format(auc_test))

accuracy of sag : 0.8923664122137405
----------------------------------------------------------------------------------------------------
f1 of sag : 0.9350605848458983
----------------------------------------------------------------------------------------------------
auc of sag : 0.8570700277191993
accuracy_test of sag : [0.8943306619174829]
----------------------------------------------------------------------------------------------------
f1_test of sag : [0.9358327325162221]
----------------------------------------------------------------------------------------------------
auc_test of sag : [0.8523562929064841]


---

#### feature_importances

In [270]:
#지수표현식 삭제
pd.options.display.float_format = '{:.5f}'.format

rdmforest_clf_morph.feature_importances_

array([3.22041158e-03, 2.36044004e-05, 2.25947600e-05, ...,
       1.63894544e-05, 0.00000000e+00, 5.26284033e-05])

In [271]:
vectorizer_tfidf_morph.get_feature_names_out()

array(['10', '100', '10000원', ..., '힘들어요', '힘들었는데', '힘듭니다'], dtype=object)

In [272]:
#지수표현식 삭제
pd.options.display.float_format = '{:.5f}'.format

#feature_importances
df_fture_importn = pd.DataFrame(rdmforest_clf_morph.feature_importances_, index=vectorizer_tfidf_morph.get_feature_names_out())
df_fture_importn = df_fture_importn.rename(columns={0:'feature_importances'})
df_fture_importn = df_fture_importn.sort_values(by = 'feature_importances', ascending=False)
df_fture_importn.head(40)

,feature_importances
업데이트,0.08534
장바구니,0.05531
오류,0.05127
가입,0.04353
문의,0.03128
광고,0.03042
삭제,0.02256
결제,0.02174
했는데,0.01770
설치,0.01503


#### feature_importances는 부호구분이 안돼서, 해당 단어가 긍정 or 부정 리뷰 중 어느 리뷰에 영향력이 있는지 알 수 없다.

---

#### 새로운 리뷰 생성
#### 문장이 말이 되는지 않되는지는 상관이 없다.(벡터화는 문장의 단어 순서를 고려하지 않고 빈도수만 고려하기 때문이다.)

In [14]:
df_keywords_morph = pd.read_csv('keywords_morph_l2_7.csv').set_index(keys='Unnamed: 0', drop=True)
df_keywords_morph

,coefficient
Unnamed: 0,
편하고,4.584159
좋은,4.415472
만족합니다,4.266957
편해요,4.197477
사용,3.930048
...,...
업데이트,-3.990138
안됨,-4.034324
문의,-4.069475


In [15]:
#positive coefficient
df_keywords_morph_pos = df_keywords_morph[df_keywords_morph['coefficient'] > 0]
df_keywords_morph_pos

,coefficient
Unnamed: 0,
편하고,4.584159
좋은,4.415472
만족합니다,4.266957
편해요,4.197477
사용,3.930048
...,...
용량,0.011167
종종,0.010485
적립,0.008004


In [16]:
#positive words list
positive_words = list(df_keywords_morph_pos.index)
positive_words

['편하고',
 '좋은',
 '만족합니다',
 '편해요',
 '사용',
 '좋습니다',
 '최고',
 '편리하고',
 '신선하고',
 '있어',
 '감사합니다',
 '아주',
 '편하게',
 '빠르고',
 '애용',
 '항상',
 '좋고',
 '편리합니다',
 '빠른',
 '없이',
 '깔끔하고',
 '클릭',
 '들어가',
 '만족',
 '있어서',
 '만족해요',
 '단점',
 '마음',
 '편리해요',
 '신선한',
 '쉽게',
 '할인',
 '환경',
 '신세계',
 '정말',
 '간편하고',
 '자주',
 '편함',
 '소량',
 '편리하게',
 '한눈',
 '신선해요',
 '그동안',
 '포장',
 '조회',
 '중독',
 '편하네요',
 '좋음',
 '빨라요',
 '다양해서',
 '좋아서',
 '늘려주세요',
 '편리한',
 '쿠폰',
 '보는',
 '맛있어요',
 '감사',
 '생활',
 '높은',
 '에만',
 '잃지',
 '편리',
 '들어요',
 '기대',
 '아닌데',
 '보다',
 '감동',
 '위로',
 '편리함',
 '요즘',
 '장보기',
 '먹거리',
 '신선해서',
 '고맙습니다',
 '맛집',
 '쓰고',
 '특히',
 '느낌',
 '있지만',
 '코로나',
 '모르는',
 '편해서',
 '직관',
 '비싸긴',
 '다양한',
 '저렴하고',
 '완전히',
 '편리하네요',
 '샛별',
 '혜택',
 '샀어요',
 '걱정',
 '등록',
 '간편해요',
 '번창',
 '좋았습니다',
 '요리',
 '간편하게',
 '편합니다',
 '그런',
 '받아',
 '깔끔한',
 '좋다',
 '보기',
 '라면',
 '되면',
 '가족',
 '온라인',
 '담아',
 '빨리',
 '터치',
 '신선하게',
 '처음',
 '매우',
 '카테고리',
 '컬리',
 '싶어요',
 '이에요',
 '인기',
 '낱개',
 '이용',
 '감사해요',
 '저녁',
 '가기',
 '한지',
 '신선도',
 '재료',
 '상당히',
 '유용해요',
 '새

In [17]:
#positive_words에서 중복을 허용해서 40개 랜덤 뽑아서 문장 만들기(8000개)
pos_sentences = {}

for i in range(8000): 
    pos_sentence = [random.choice(positive_words) for i in range(40)]
    
    pos_sentences[' '.join(pos_sentence)] = 1

In [18]:
pos_sentences

{'만족스러워요 디자인 오면 팔아서 똑같은 아닌데 맛있어서 신선하게 하러 백원 까진 활용 계란 질도 전용 신세계 그동안 요즘 대체로 알뜰 만들어 걱정 정상 빠릅니다 먹거리 아무튼 마니 으로도 이다 라면 가지 스크롤 얼음 이에요 비싸긴 만족합니다 사이 쿠폰 당일 제외': 1,
 '되니까 다양한 하기도 나름 해주시 종종 주소지 그런 시골 기재 저렴하게 마콜 있으니 감동 나중 전반 관련 편해서 일정 오면 주로 해당 너무나 감동 있다는 많고요 에만 많으면 컬리 엄마 정말 이제야 맞게 받아 친환경 좋을 괜찮음 새벽 유용합니다 다양해요': 1,
 '있습니다 빠르고 아쉽지만 지속 조회 전이 매력 꾸준히 들어가서 똑같이 아닌 들어가서 않았는데 베스트 송도 들어가서 봤는데 새로 아직 모아 오면 됐는데 싶은 좋을듯 없어요 에만 완전히 저만 시도 빼면 하다가 한눈 시키면 새로운 1년 주세요 합리 친환경 주소지 편입': 1,
 '아무튼 용량 있는데요 모아서 간단 빠릅니다 원하는 글자 그런 안되는 로서 장보고 깔끔하고 배송 키트 바로바로 싶은 편리하게 꼼꼼하고 제거 빼면 메뉴 껐다 싱싱해요 아래 마니 됩니다 한지 많다 단계 만큼 하시는 글자 고맙습니다 와요 시켜도 할인 선별 있지 해주셔서': 1,
 '지는 하려고요 싱싱 후회 바쁜 유용합니다 이유식 감사한 깔끔하게 조회 비쌀 있었습니다 아닌가 보면서 담날 보는 다양하게 않는데 우유 고급 가게 새벽 즐거움 모르는 깔끔해서 아닌 확대 안전하게 보는데 역시 즐겨찾기 보다 가게 원하는 괜찮고 감사하겠습니다 로켓 좋은 맞게 간편하고': 1,
 '좋다고 좋을 추합니다 주셨으면 선물 완전 할듯 강추 보기 다양해졌으면 올려주세요 편합니다 깔고 간단 종종 보러 숫자 없으니 빠름 이점 수량 가장 발전 살수 장보고 이다 매제 편해서 가요 싶네요 들어가도 사과 되었습니다 만족스럽습니다 얼음 나오네요 겁니다 말고 약속 빨라요': 1,
 '필수 사과 않나요 즐거움 행복해요 언제나 로서 맛볼 샀어요 또는 해라 너무 눌러도 5천 아주 아주아주 좋겠습니다 상단 강

In [19]:
#새로운 positive 리뷰
df_review_new_pos = pd.DataFrame(pd.Series(pos_sentences)).reset_index().rename(columns={'index': 'review', 0:'label'})
df_review_new_pos

,review,label
0,만족스러워요 디자인 오면 팔아서 똑같은 아닌데 맛있어서 신선하게 하러 백원 까진 활...,1
1,되니까 다양한 하기도 나름 해주시 종종 주소지 그런 시골 기재 저렴하게 마콜 있으니...,1
2,있습니다 빠르고 아쉽지만 지속 조회 전이 매력 꾸준히 들어가서 똑같이 아닌 들어가서...,1
3,아무튼 용량 있는데요 모아서 간단 빠릅니다 원하는 글자 그런 안되는 로서 장보고 깔...,1
4,지는 하려고요 싱싱 후회 바쁜 유용합니다 이유식 감사한 깔끔하게 조회 비쌀 있었습니...,1
...,...,...
7995,야채 최고 8시 빠릅니다 했었는데 있었습니다 거리 빼면 카테고리 돼있어서 되길 가게...,1
7996,만족하고 바쁜 최근 기는 있어도 감사해요 할듯 하여 급할 지속 사용성 간식 직장 만...,1
7997,시켜도 정상 오고 가도 이제 수고 일찍 제거 제거 들어가는 할게요 단계 빼면 선물 ...,1
7998,지인 아침 편하게 새로운 있긴 활용 부탁드려요 방법 있는 않는데 과정 반복 편하고 ...,1


In [20]:
#negative coefficient
df_keywords_morph_neg = df_keywords_morph[df_keywords_morph['coefficient'] < 0]
df_keywords_morph_neg

,coefficient
Unnamed: 0,
된다는,-0.000915
하시길,-0.001385
가능하게,-0.003879
있나요,-0.004121
기회,-0.004310
...,...
업데이트,-3.990138
안됨,-4.034324
문의,-4.069475


In [21]:
#negative words list
negative_words = list(df_keywords_morph_neg.index)
negative_words

['된다는',
 '하시길',
 '가능하게',
 '있나요',
 '기회',
 '넣어도',
 '소리',
 '원래',
 '중간',
 '비용',
 '카카오',
 '신청',
 '현관',
 '발급',
 '보이는',
 '건의',
 '씁니다',
 '식사',
 '개수',
 '에서는',
 '물량',
 '7월',
 '근데',
 '신규',
 '드라이아이스',
 '유효',
 '이니',
 '검색',
 '포인트',
 '하겠어요',
 '들어서',
 '된다면',
 '걸리고',
 '많이',
 '하길',
 '캐시',
 '사이트',
 '거래',
 '입고',
 '응대',
 '어차피',
 '느낄',
 '4시',
 '결정',
 '고기',
 '현재',
 '리지',
 '다양했으면',
 '식료품',
 '비교',
 '일도',
 '한다',
 '깨끗하게',
 '수십',
 '통장',
 '그런지',
 '다가',
 '평소',
 '줘요',
 '체크',
 '아무래도',
 '항목',
 '하니',
 '좋았고',
 '최신',
 '분들',
 '봅니다',
 '다시',
 '한테',
 '노력',
 '메인',
 '처럼',
 '제일',
 '체적',
 '이상한',
 '해주세요',
 '절약',
 '받았어요',
 '평가',
 '목록',
 '낮은',
 '가능',
 '정신',
 '옵니다',
 '내용물',
 '딱히',
 '보면',
 '쓰는',
 '편한',
 '확장',
 '방식',
 '홈페이지',
 '송장',
 '10시',
 '오는',
 '남깁니다',
 '멈추고',
 '냉동식품',
 '들어갈',
 '남겨요',
 '건데',
 '경험',
 '좋은지',
 '오프라인',
 '인가',
 '있답니다',
 '만드는',
 '제발',
 '달이',
 '설정',
 '유기농',
 '보고',
 '개월',
 '어쩔',
 '오전',
 '이미',
 '댓글',
 '상황',
 '이네요',
 '준다고',
 '좋겠네요',
 '리스트',
 '나게',
 '많네요',
 '많을',
 '이란',
 '눌러',
 '크롤',
 '변경',
 '봤지만',
 '많습니다',
 '바랍니다',
 '하게',
 

In [22]:
#negative_words에서 중복을 허용해서 50개 랜덤 뽑아서 문장 만들기(2000개)
neg_sentences = {}

for i in range(2000): 
    neg_sentence = [random.choice(negative_words) for i in range(50)]
    
    neg_sentences[' '.join(neg_sentence)] = 0

In [23]:
neg_sentences

{'몰라도 하면 홈페이지 포인트 구글 오는데 갑니다 물품 있게 모르겠지만 때문 고생 그대로 거임 다양성 있고요 사는 준다는 많이 냉동 이메일 입장 나면 10분 똑바로 물품 이라고 싶지 갑니다 나는데 들어갈 다시 제주도 슈퍼 유저 남겨요 만들고 딱히 많습니다 하시고 없다고 유기농 해놓고 왔네요 브랜드 일반 냉동 오전 비싸고 개도': 0,
 '6시 이랑 사려 의견 리스트 페이 싶은데 느낄 어차피 열심히 해도 11시 에서는 놓고 나옴 하시길 분들 받은 있게 조치 라는 싶은데 어제 이란 비싸고 기사 사라 마다 오니 으로는 없다고 안정 비었다고 이니 미리 괜찮은 상담사 되는데 알아서 대가 중단 으로는 아파트 최악 들어갈 가는 비교 비었다고 수가 바랍니다': 0,
 '잘못 안되니 나는데 약관 발송 상술 담아도 나는데 없는데 안주 치고 버리고 않지만 좋겠지만 사는 한국 같아서 눌러 주면 상품 보여요 하세요 강제 하신 나는데 전국 아파트 이란 식료품 일시 나요 싶지 네트워크 업데이트 점점 담당 않지만 주소 선택 크게 듭니 이용자 하려면 수정 한국 유도 담당자 치면 오류로 작동': 0,
 '10 없다는 휴대폰 대로 오전 일반 초대 안정 아니라 필요할 다양하지 부분 바랍니다 단위 별도 예전 조건 솔직히 걸려요 타사 되죠 뜨고 카드 이라고 스럽게 스럽게 그렇게 상담사 있게 인지 가는 속도 시중 똑같아요 시중 뜨는데 비쌈 맘대로 뜨면서 손쉽게 정도 나요 정신 절약 힘든데 이었는데 담고 시간대 의견 정확하게': 0,
 '의견 무지 파세요 들어 종류 먹기 이라서 댓글 장난 들어 아이디 아니라 결재 두고 양해 쓰기 소리 했네요 스마일 자리 않습니다 불편하네요 치고 됐다고 가능하게 준다 안정 똑같아요 많아졌으면 다양했으면 부탁 물건 받았는데 해놓고 해야 누르고 사려 1시간 입장 최저 홈페이지 체크 자체 봤지만 초심 맞지 줘요 많네요 이동 수가': 0,
 '안녕하세요 만들어주세요 아닌가요 없는데 거야 홍보 쓰레기 읽고 지금 종료 하단 된다면 하기가 줘요 아니네요 확실히 메일 놓고 하단 오는

In [24]:
#새로운 negative 리뷰
df_review_new_neg = pd.DataFrame(pd.Series(neg_sentences)).reset_index().rename(columns={'index': 'review', 0:'label'})
df_review_new_neg

,review,label
0,몰라도 하면 홈페이지 포인트 구글 오는데 갑니다 물품 있게 모르겠지만 때문 고생 그...,0
1,6시 이랑 사려 의견 리스트 페이 싶은데 느낄 어차피 열심히 해도 11시 에서는 놓...,0
2,잘못 안되니 나는데 약관 발송 상술 담아도 나는데 없는데 안주 치고 버리고 않지만 ...,0
3,10 없다는 휴대폰 대로 오전 일반 초대 안정 아니라 필요할 다양하지 부분 바랍니다...,0
4,의견 무지 파세요 들어 종류 먹기 이라서 댓글 장난 들어 아이디 아니라 결재 두고 ...,0
...,...,...
1995,관리 비닐 약관 진행 같은 사야 배달 상세 일부러 제도 왔네요 이란 않아요 된다 받...,0
1996,썼는데 없다는 상술 많아 상황 아예 하루 연결 모르겠어요 계속 하는지 있다고 카카오...,0
1997,위치 버림 있었어요 매일 전혀 비번 편한 급하게 그러네요 오늘 건가 된다면 그냥 이...,0
1998,귀찮아서 보지 하네 안된다는 지우고 접근 이든 옵니다 해야지 싶은데 인해 하라 거야...,0


In [25]:
#새로운 positive negative 리뷰 합치기
df_review_new = pd.concat([df_review_new_pos, df_review_new_neg])
df_review_new

,review,label
0,만족스러워요 디자인 오면 팔아서 똑같은 아닌데 맛있어서 신선하게 하러 백원 까진 활...,1
1,되니까 다양한 하기도 나름 해주시 종종 주소지 그런 시골 기재 저렴하게 마콜 있으니...,1
2,있습니다 빠르고 아쉽지만 지속 조회 전이 매력 꾸준히 들어가서 똑같이 아닌 들어가서...,1
3,아무튼 용량 있는데요 모아서 간단 빠릅니다 원하는 글자 그런 안되는 로서 장보고 깔...,1
4,지는 하려고요 싱싱 후회 바쁜 유용합니다 이유식 감사한 깔끔하게 조회 비쌀 있었습니...,1
...,...,...
1995,관리 비닐 약관 진행 같은 사야 배달 상세 일부러 제도 왔네요 이란 않아요 된다 받...,0
1996,썼는데 없다는 상술 많아 상황 아예 하루 연결 모르겠어요 계속 하는지 있다고 카카오...,0
1997,위치 버림 있었어요 매일 전혀 비번 편한 급하게 그러네요 오늘 건가 된다면 그냥 이...,0
1998,귀찮아서 보지 하네 안된다는 지우고 접근 이든 옵니다 해야지 싶은데 인해 하라 거야...,0


Step1. Random foreset model도 Logistic Regression model training에 사용한 실제 리뷰 데이터로 training을 한다. <br>
Step2. Logistic Regression의 coefficiet가 양수(+)인 feature(키워드)로 새로운 리뷰를 생성하고 1 label을 붙였다. <br>
Step3. Logistic Regression의 coefficiet가 음수(-)인 feature(키워드)로 새로운 리뷰를 생성하고 0 label을 붙였다. <br>
Step3. Random foreset가 Logistic Regression의 feature들로 생성된 리뷰를 잘 분류하는지 확인한다.

In [26]:
#새로운 리뷰들 벡터화 후 예측

#skfold
skfold = StratifiedKFold(n_splits=5)

#best randomforest
rdmforest_clf_morph = RandomForestClassifier(max_depth=35, 
                                             max_features=0.1, 
                                             min_samples_leaf=1, 
                                             min_samples_split=6, 
                                             n_estimators=500)


#train(fit)은 기존의 review data로 한다.
for train_idx, validation_idx in skfold.split(X_morph_train, y=y_morph_train):
        
    X_train = X_morph_train[train_idx]
    X_validation = X_morph_train[validation_idx]
        
    y_train = y_morph_train.iloc[train_idx]
    y_validation = y_morph_train.iloc[validation_idx]
        
    #fit_predict
    rdmforest_clf_morph.fit(X_train, y_train)
    
    
#새로운 리뷰 test data
contents_for_vectorize_new = df_review_new['review']


X_test_new = vectorizer_tfidf_morph.fit_transform(contents_for_vectorize_new)
y_test_new = df_review_new['label']


#새로운 리뷰 predict
pred_test_new = rdmforest_clf_morph.predict(X_test_new)


#test_data accuracy, f1, auc
accuracy_test.append(accuracy_score(pred_test_new, y_test_new))
f1_test.append(f1_score(pred_test_new, y_test_new))
auc_test.append(roc_auc_score(pred_test_new, y_test_new))

print('실제 리뷰 test_data의 accuracy: {}'.format(accuracy_test[0]))
print('새로운 리뷰의 accuracy: {}'.format(accuracy_test[1]))
print('-'*100)
print('실제 리뷰 test_data의 f1: {}'.format(f1_test[0]))
print('새로운 리뷰의 f1: {}'.format(f1_test[1]))
print('-'*100)
print('실제 리뷰 test_data의 auc_test: {}'.format(auc_test[0]))
print('새로운 리뷰의 auc_test: {}'.format(auc_test[1]))

실제 리뷰 test_data의 accuracy: 0.8943306619174829
새로운 리뷰의 accuracy: 0.885
----------------------------------------------------------------------------------------------------
실제 리뷰 test_data의 f1: 0.9358327325162221
새로운 리뷰의 f1: 0.9297495418448382
----------------------------------------------------------------------------------------------------
실제 리뷰 test_data의 auc_test: 0.8523562929064841
새로운 리뷰의 auc_test: 0.8349678592108832


#### Before
#### 실제 리뷰 데이터로 fit을 하고, 새롭게 생성한 리뷰로 predict을 하는 것은 실행이 안된다. 
#### 실제 리뷰 데이터를 벡터화해서 생성된 feature와 새로운 리뷰를 벡터화해서 생성된 feature의 개수가 다르기 때문이다.
#### 그렇다고 새롭게 생성된 리뷰로 fit하고 predict하는 것은 그냥 학습한 것으로 예측하는 것이기 때문에 의미가 없다.

#### After
#### 새롭게 생성하는 리뷰의 길이를 늘려 충분한 feature 개수 확보 후, predict를 실행하면 가능하다.

---

In [23]:
pred_test_new

array([1., 1., 1., ..., 1., 0., 0.])

In [24]:
y_test_new

0       1
1       1
2       1
3       1
4       1
       ..
1995    0
1996    0
1997    0
1998    0
1999    0
Name: label, Length: 10000, dtype: int64

In [25]:
#y_test_new label과 예측 결과를 dataframe으로 변환
df_rdmforst = y_test_new.to_frame()
df_rdmforst['pred'] = pred_test_new
df_rdmforst['TF'] = df_rdmforst['label'] == df_rdmforst['pred']
df_rdmforst

,label,pred,TF
0,1,1.0,True
1,1,1.0,True
2,1,1.0,True
3,1,1.0,True
4,1,1.0,True
...,...,...,...
1995,0,1.0,False
1996,0,0.0,True
1997,0,1.0,False
1998,0,0.0,True


In [26]:
#label과 prediction이 일치한 결과
df_right = df_rdmforst[df_rdmforst['TF']]
df_right

,label,pred,TF
0,1,1.0,True
1,1,1.0,True
2,1,1.0,True
3,1,1.0,True
4,1,1.0,True
...,...,...,...
1993,0,0.0,True
1994,0,0.0,True
1996,0,0.0,True
1998,0,0.0,True


In [29]:
#1을 맞춘 비율(1의 accuracy)
len(df_right[df_right['pred'] == 1])/ 8000

0.9285

In [30]:
#0을 맞춘 비율(0의 accuracy)
len(df_right[df_right['pred'] == 0])/ 2000

0.5825

#### 0에 대한 정확도가 현저하게 떨어진다.

----

긍정리뷰와 부정리뷰를 각각 동일한 개수를 생성한다.

In [31]:
#positive_words에서 중복을 허용해서 5개 랜덤 뽑아서 문장 만들기(3000개)
pos_sentences_same = {}

for i in range(6000): 
    pos_sentence_same = [random.choice(positive_words) for i in range(6)]
    
    pos_sentences_same[' '.join(pos_sentence_same)] = 1

In [32]:
#새로운 positive 리뷰
df_review_new_pos_same = pd.DataFrame(pd.Series(pos_sentences_same)).reset_index().rename(columns={'index': 'review', 0:'label'})
df_review_new_pos_same

,review,label
0,신선함 간식 만큼 보관 인터페이스 다이어트,1
1,나이스 되었습니다 같아요 보면서 인터페이스 줘서,1
2,맛볼 최애 진작 중이 올려주세요 빨라요,1
3,칭찬 사면 이사 건강한 저렴하고 보내주셔서,1
4,직장 특정 행사 받기 고려 감사,1
...,...,...
5995,까진 시켜서 먹는 싶을 있어서 정말,1
5996,같고 가성 맞게 좋아서 이라 신선해서,1
5997,정확한 드는 깔아서 새로운 시도 되었다고,1
5998,많고 마인드 버튼 가기 불편한 이용,1


In [33]:
#negative_words에서 중복을 허용해서 5개 랜덤 뽑아서 문장 만들기(3000개)
neg_sentences_same = {}

for i in range(4000): 
    neg_sentence_same = [random.choice(negative_words) for i in range(6)]
    
    neg_sentences_same[' '.join(neg_sentence_same)] = 0

In [34]:
#새로운 positive 리뷰
df_review_new_neg_same = pd.DataFrame(pd.Series(neg_sentences_same)).reset_index().rename(columns={'index': 'review', 0:'label'})
df_review_new_neg_same

,review,label
0,나옵니다 깔았는데 블랙리스트 차별 예전 구분,0
1,거야 마감 근데 지나도 오전 대가,0
2,불편합니다 하단 비쌈 경험 똑바로 정작,0
3,갤럭시 다양하지 달이 낚시 좋겠네요 설정,0
4,작성 중간 오픈 10 무지 지연,0
...,...,...
3995,대로 파는 가고 이전 옵니다 있던,0
3996,대기 뜨고 시장 하죠 운영 없나요,0
3997,몰라도 으로는 아예 개도 하세요 떠요,0
3998,확실히 지우고 분들 고민 적립금 필요할,0


In [35]:
#새로운 positive negative 리뷰 합치기
df_review_new_same = pd.concat([df_review_new_pos_same, df_review_new_neg_same])
df_review_new_same

,review,label
0,신선함 간식 만큼 보관 인터페이스 다이어트,1
1,나이스 되었습니다 같아요 보면서 인터페이스 줘서,1
2,맛볼 최애 진작 중이 올려주세요 빨라요,1
3,칭찬 사면 이사 건강한 저렴하고 보내주셔서,1
4,직장 특정 행사 받기 고려 감사,1
...,...,...
3995,대로 파는 가고 이전 옵니다 있던,0
3996,대기 뜨고 시장 하죠 운영 없나요,0
3997,몰라도 으로는 아예 개도 하세요 떠요,0
3998,확실히 지우고 분들 고민 적립금 필요할,0


In [36]:
#새로운 리뷰들 벡터화 후 예측

#skfold
skfold = StratifiedKFold(n_splits=5)

#best randomforest
rdmforest_clf_morph = RandomForestClassifier(max_depth=35, 
                                             max_features=0.1, 
                                             min_samples_leaf=1, 
                                             min_samples_split=6, 
                                             n_estimators=400)


#train(fit)은 기존의 review data로 한다.
for train_idx, validation_idx in skfold.split(X_morph_train, y=y_morph_train):
        
    X_train = X_morph_train[train_idx]
    X_validation = X_morph_train[validation_idx]
        
    y_train = y_morph_train.iloc[train_idx]
    y_validation = y_morph_train.iloc[validation_idx]
        
    #fit_predict
    rdmforest_clf_morph.fit(X_train, y_train)
    
    
#새로운 리뷰 test data
contents_for_vectorize_new_same = df_review_new_same['review']


X_test_new_same = vectorizer_tfidf_morph.fit_transform(contents_for_vectorize_new_same)
y_test_new_same = df_review_new_same['label']


#새로운 리뷰 predict
pred_test_new_same = rdmforest_clf_morph.predict(X_test_new_same)


#test_data accuracy, f1, auc
accuracy_test.append(accuracy_score(pred_test_new_same, y_test_new_same))
f1_test.append(f1_score(pred_test_new_same, y_test_new_same))
auc_test.append(roc_auc_score(pred_test_new_same, y_test_new_same))

In [38]:
accuracy_test

[0.8919560700504601, 0.8593, 0.6208]

In [39]:
f1_test

[0.9343671114316624, 0.9134845969378345, 0.7570476678626346]

In [40]:
auc_test

[0.8474663196570729, 0.7848218584891024, 0.6901051844551309]

In [41]:
#y_test_new_same label과 예측 결과를 dataframe으로 변환
df_rdmforst_same = y_test_new_same.to_frame()
df_rdmforst_same['pred'] = pred_test_new_same
df_rdmforst_same['TF'] = df_rdmforst_same['label'] == df_rdmforst_same['pred']
df_rdmforst_same

,label,pred,TF
0,1,1.0,True
1,1,1.0,True
2,1,1.0,True
3,1,1.0,True
4,1,1.0,True
...,...,...,...
3995,0,1.0,False
3996,0,1.0,False
3997,0,1.0,False
3998,0,1.0,False


In [42]:
#label과 prediction이 일치한 결과
df_right_same = df_rdmforst_same[df_rdmforst_same['TF']]
df_right_same

,label,pred,TF
0,1,1.0,True
1,1,1.0,True
2,1,1.0,True
3,1,1.0,True
4,1,1.0,True
...,...,...,...
3908,0,0.0,True
3914,0,0.0,True
3954,0,0.0,True
3988,0,0.0,True


In [43]:
#0을 맞춘 비율(0의 accuracy)
len(df_right_same[df_right_same['pred'] == 0])/ 1000

0.3

???? 오히려 더 떨어짐